# File description
Third step in the preprocessing. This file takes the compressed videos from `reducer2_h265.ipynb` crop each video individually. That is to say, each video is manually inspected and some suitable cropping values are chosen for that particular video. The cropping is performed with the help of `ffmpeg`

In [ ]:
import dutils as U
U.jupyter_ipython.adjust_screen_width()
from glob import glob
import os
import torch
import random
import cv2
from tqdm.notebook import tqdm
import subprocess
import pandas as pd

# Constants

In [ ]:
OLD_FOLDER_PATH = "C:\\Users\\JK\\Desktop\\reduced_h265" # Path to the original videos
NEW_FOLDER_PATH = "C:\\Users\\JK\\Desktop\\reduced_crop" # Path where the processed videos are to be saved

# Finding cropping values
Take a single frame from each day which is going to be used for determining the cropping values

In [ ]:
frames = {}
videos_already_done = [os.path.basename(p) for p in glob(os.path.join(NEW_FOLDER_PATH, "*"))]

for folder_path in glob(OLD_FOLDER_PATH+"\\*"):
    if os.path.basename(folder_path) in videos_already_done:
        continue
    for i, video_path in enumerate(sorted(glob(os.path.join(folder_path, "*")), key=os.path.getctime)):
        if i < 30: continue # Don't want the dark morning images
        cap = cv2.VideoCapture(video_path)
        _, frame = cap.read()
        cap.release()
        cv2.destroyAllWindows()
        frames[folder_path] = frame
        break
        
U.jupyter_ipython.show_image(list(frames.values()))

In [ ]:
results = {}

def testing_cropping_values(image, x1, y1, x2, y2):
    """ 
    A simple function used to iterartively figure out 
    which cropping values are suitable for each image.
    The reformatting to width (x2-x1) and height (y2-y1)
    are because that's the format FFMPEG expects
    """
    H, W, _ = image.shape
    if x2 is None: x2 = W
    if x2 < 0: x2 = W + x2
    if y2 is None: y2 = H
    if y2 < 0: y2 = H + y2
    
    cropped_image = image[y1:y2,x1:x2,:]
    #U.images.show_ndarray_image(image, 0.5, BGR2RGB=True)
    U.images.show_ndarray_image(cropped_image, 0.75, BGR2RGB=True)
    print(x1,y1,x2,y2)
    return (x1, y1, x2-x1, y2-y1)

for i, (path, frame) in enumerate(frames.items()):
    if i == 0:
        results[path] = testing_cropping_values(frames[path],x1=0, y1=350, x2=None, y2=-50)
    elif i == 1:
        results[path] = testing_cropping_values(frames[path],x1=0, y1=350, x2=None, y2=-50)
    # ...
    # NOTE ALL VIDEOS WAS SEND THROUGH THIS PROCESS, BUT ONLY THE LAST 2 VALUES ARE SAVED HERE
    # ...

path_to_crop_values = results

# Folders

In [ ]:
old_folder_paths = [os.path.abspath(p) for p in glob(os.path.join(OLD_FOLDER_PATH, "*"))]
new_folder_paths = [os.path.join(NEW_FOLDER_PATH, os.path.basename(p)) for p in old_folder_paths]

for new_folder_path in new_folder_paths:
    if os.path.exists(new_folder_path): continue
    assert not os.path.exists(new_folder_path), "Folder already exists"
    assert not os.path.isdir(new_folder_path), "Received non-folder path"
    os.mkdir(new_folder_path)

# Compress

In [ ]:
for (old_folder_path, new_folder_path) in tqdm(list(zip(old_folder_paths, new_folder_paths))):
    if os.path.basename(old_folder_path) in videos_already_done:
        continue
    x,y,w,h = path_to_crop_values[old_folder_path]
    
    for old_video_path in tqdm(glob(os.path.join(old_folder_path, "*")), leave=False):
        new_video_path = os.path.join(new_folder_path, os.path.basename(old_video_path))
        if os.path.isfile(new_video_path): continue
        command = f'ffmpeg -i {old_video_path} -filter:v "crop={w}:{h}:{x}:{y}" {new_video_path}'
        subprocess.run(command)